In [71]:
import pandas as pd
import math as mt
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('data-mhs.csv', sep=';')
df = pd.DataFrame(data)
df.style.hide_index()

Nama,Jenis Kelamin,IPK,Penghasilan Orangtua,Alamat,Prestasi
Ali,L,3.4,3000000,Sumenep,Internasional
Ani,P,3.2,5000000,Surabaya,Regional
Abi,L,3.3,4000000,Bangkalan,Nasional


In [72]:
X = data.iloc[:,:].values
labelEncode_X = LabelEncoder()
X[:,1] = labelEncode_X.fit_transform(X[:,1])

def Zscore(x,mean,std):
    top = x - mean
    if top==0:
        return top
    else:
        return round(top / std, 2)

In [73]:
#menghitung jarak tipe numerikal
def euclidianDistance(x,y):
    dis = 0
    for i in range(len(x)):
        dis += (x[i] - y[i]) ** 2
    return round(mt.sqrt(dis),2)

def normalisasi(num, col_x): 
    return Zscore(num, pd.Series(data[col_x].values).mean(), pd.Series(data[col_x].values).std())

In [74]:
#Menghitung Jarak tipe categorikal
def distanceNom(x,y):
    p = len(x) or len(y)
    m = 0
    for i in range(len(x)):
        if x[i][0] == y[i][0]:
            m +=1
    return (p - m) / p


In [75]:
#Menghitung Jarak tipe ordinal
#inisialisasi
x = {'Internasional':3,'Nasional':2,'Regional':1}
def normalizedOrd(y):
    i_max = 0
    for i in x:
        if x[i] > i_max:
            i_max = x[i]
        if y[0] == i:
            i_val = x[i]
    return (i_val - 1) / (i_max - 1)



In [82]:
#Menghitung jarak tipe binary
def distanceSimetris(x,y):
    q=r=s=t=0
    for i in range(len(x)):
        if x[i]==1 and y[i]==1:
            q+=1
        elif x[i]==1 and y[i]==0:
            r+=1
        elif x[i]==0 and y[i]==1:
            s+=1
        elif x[i]==0 and y[i]==0:
            t+=1
    return ((r+s)/(q+r+s+t))


d_x = {
    0 : ['', 'Ali', 'Ani', 'Abi'],
    1 : ['Ali', 0, '', ''],
    2 : ['Ani', '', 0, ''],
    3 : ['Abi', '', '', 0]
}

In [83]:
#ambil data numerikal
aliNum = df.iloc[0, 2:4].values
aniNum = df.iloc[1, 2:4].values
abiNum = df.iloc[2, 2:4].values
#normalisasi data numerikal
aliNum = [normalisasi(aliNum[0], data.columns[2]), normalisasi(aliNum[1], data.columns[3])]
aniNum = [normalisasi(aniNum[0], data.columns[2]), normalisasi(aniNum[1], data.columns[3])]
abiNum = [normalisasi(abiNum[0], data.columns[2]), normalisasi(abiNum[1], data.columns[3])]

d_x[1][2] = euclidianDistance(aniNum,aliNum)
d_x[1][3] = euclidianDistance(abiNum,aliNum)
d_x[2][3] = euclidianDistance(abiNum,aniNum)

d_x = pd.DataFrame(d_x)
d_x.style.hide_index()

0,1,2,3
,Ali,Ani,Abi
Ali,0,,
Ani,2.83,0,
Abi,1.41,1.41,0


In [84]:
#ambil data kategorical
aliKat = [df.iloc[0, 4:5].values]
aniKat = [df.iloc[1, 4:5].values]
abiKat = [df.iloc[2, 4:5].values]

d_x[1][2] = distanceNom(aniKat,aliKat)
d_x[1][3] = distanceNom(abiKat,aliKat)
d_x[2][3] = distanceNom(abiKat,aniKat)

d_x = pd.DataFrame(d_x)
d_x.style.hide_index()

0,1,2,3
,Ali,Ani,Abi
Ali,0,,
Ani,1,0,
Abi,1,1,0


In [79]:
#ambil data binary
aliBin = X[0, 1:2]
aniBin = X[1, 1:2]
abiBin = X[2, 1:2]

d_x[1][2] = distanceSimetris(aniBin,aliBin)
d_x[1][3] = distanceSimetris(abiBin,aliBin)
d_x[2][3] = distanceSimetris(abiBin,aniBin)

d_x = pd.DataFrame(d_x)
d_x.style.hide_index()

0,1,2,3
,Ali,Ani,Abi
Ali,0,,
Ani,1,0,
Abi,0,1,0


In [85]:
#ambil data ordinal
aliOrd = [df.iloc[0, 5:6].values]
aniOrd = [df.iloc[1, 5:6].values]
abiOrd = [df.iloc[2, 5:6].values]

d_x[1][2] = euclidianDistance([normalizedOrd(aniOrd)],[normalizedOrd(aliOrd)])
d_x[1][3] = euclidianDistance([normalizedOrd(abiOrd)],[normalizedOrd(aliOrd)])
d_x[2][3] = euclidianDistance([normalizedOrd(abiOrd)],[normalizedOrd(aniOrd)])

d_x = pd.DataFrame(d_x)
d_x.style.hide_index()

0,1,2,3
,Ali,Ani,Abi
Ali,0,,
Ani,1,0,
Abi,0.5,0.5,0


In [86]:
d_x[1][2] = euclidianDistance(aniNum,aliNum) + \
            distanceNom(aniKat,aliKat) + distanceSimetris(aniBin,aliBin) +\
            euclidianDistance([normalizedOrd(aniOrd)],[normalizedOrd(aliOrd)])
d_x[1][3] = euclidianDistance(abiNum,aliNum) + \
            distanceNom(abiKat,aliKat) + distanceSimetris(abiBin,aliBin) +\
            euclidianDistance([normalizedOrd(abiOrd)],[normalizedOrd(aliOrd)])
d_x[2][3] = euclidianDistance(abiNum,aniNum) + \
            distanceNom(abiKat,aniKat) + distanceSimetris(abiBin,aniBin) +\
            euclidianDistance([normalizedOrd(abiOrd)],[normalizedOrd(aniOrd)])

d_x = pd.DataFrame(d_x)
d_x.style.hide_index()

0,1,2,3
,Ali,Ani,Abi
Ali,0,,
Ani,5.83,0,
Abi,2.91,3.91,0
